### Lesson 1: Multi-Agent Conversation and Stand-up Comedy

#### Setup

In [22]:
#r "nuget:AutoGen"

using AutoGen.Core;
using AutoGen.OpenAI;
using AutoGen.OpenAI.Extension;
using Azure.AI.OpenAI;
using System.Threading;

var openAIKey = Environment.GetEnvironmentVariable("OPENAI_API_KEY") ?? throw new Exception("Please set the OPENAI_API_KEY environment variable.");
var openAIModel = "gpt-3.5-turbo";
var openaiClient = new OpenAIClient(openAIKey);


Installed Packages AutoGen, 0.0.16

Define an OpenAI Chat Agent
You can also connect to other LLM platforms like Mistral, Gemini, Ollama by using a specific agent
For example, using MistralChatAgent to connect to Mistral

In [23]:
var agent = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "chatbot",
    modelName: openAIModel)
    .RegisterMessageConnector() // convert OpenAI Message to AutoGen Message
    .RegisterPrintMessage(); // print the message to the console

In [24]:
var _ = await agent.SendAsync("Tell me a joke");

from: chatbot
Sure! Here's a joke for you:

Why don't scientists trust atoms?

Because they make up everything!



#### Setup token count middleware
// We use a token count middleware to collect all the oai messages which contain the token count information
// In the rest of examples, we use non-streaming agent because the token information is not available in streaming chunks.

In [25]:
class CountTokenMiddleware : IMiddleware
{
    private readonly List<ChatCompletions> messages = new();
    public string? Name => nameof(CountTokenMiddleware);

    public int GetTokenCount()
    {
        return messages.Sum(m => m.Usage.CompletionTokens);
    }

    public async Task<IMessage> InvokeAsync(MiddlewareContext context, IAgent agent, CancellationToken cancellationToken = default)
    {
        var reply = await agent.GenerateReplyAsync(context.Messages, context.Options, cancellationToken: cancellationToken);

        if (reply is IMessage<ChatCompletions> message)
        {
            messages.Add(message.Content);
        }

        return reply;
    }
}

var tokenCountMiddleware = new CountTokenMiddleware();

#### Conversation
Setting up a conversation between two agents, Cathy and Joe, where the memory of their interactions is retained.

In [26]:
var openaiMessageConnector = new OpenAIChatRequestMessageConnector();
var cathy = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "cathy",
    modelName: openAIModel,
    systemMessage: "Your name is Cathy and you are a stand-up comedian.")
    .RegisterMiddleware(tokenCountMiddleware) // register the token count middleware. The `RegisterMiddleware` also convert an `IStreamingAgent` to `IAgent` and block its streaming API.
    .RegisterMiddleware(openaiMessageConnector)
    .RegisterPrintMessage();

var joe = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "joe",
    modelName: openAIModel,
    systemMessage: """
    Your name is Joe and you are a stand-up comedian.
    Start the next joke from the punchline of the previous joke.
    """)
    .RegisterMiddleware(tokenCountMiddleware)
    .RegisterMiddleware(openaiMessageConnector)
    .RegisterPrintMessage();


In [27]:
var chatResult = await joe.SendAsync(
    receiver: cathy,
    message: "I'm Joe. Let's keep the jokes rolling.",
    maxRound: 4);

TextMessage from cathy
--------------------
Hey Joe! Glad you're ready for some laughs. So, I've been thinking about getting in shape. But then I remembered round is a shape, so I think I'm good for now. How about you, Joe?
--------------------

TextMessage from joe
--------------------
I'm definitely more of an oval shape myself. But hey, at least I can say I'm well-rounded!
--------------------

TextMessage from cathy
--------------------
Cathy
--------------------

TextMessage from joe
--------------------
Why did the tomato turn red? Because it saw Cathy ketchup!
--------------------



Print token consumption

In [28]:
Console.WriteLine($"Total Token count: {tokenCountMiddleware.GetTokenCount()}");

Total Token count: 85


Get a better summary of conversation from a summary agent.

In [29]:
var summaryAgent = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "summary",
    modelName: openAIModel,
    systemMessage: "You are a helpful AI assistant.")
    .RegisterMessageConnector()
    .RegisterPrintMessage();

await summaryAgent.SendAsync("summarize the converation", chatResult);

from: summary
Joe and Cathy shared some light-hearted jokes with each other. Joe joked about being oval-shaped and well-rounded, while Cathy made a joke about round being a shape. Joe also made a pun involving Cathy's name and ketchup. Overall, they had a fun and playful exchange centered around humor.



#### Chat Termination
Chat can be terminated using a termination conditions.
Terminate the conversation by running the chat step by step and check if the conversation meeting the terminate condition


In [30]:
cathy = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "cathy",
    modelName: openAIModel,
    systemMessage: """
    Your name is Cathy and you are a stand-up comedian.
    When you're ready to end the conversation, say 'I gotta go'.
    """)
    .RegisterMiddleware(tokenCountMiddleware) // register the token count middleware. The `RegisterMiddleware` also convert an `IStreamingAgent` to `IAgent` and block its streaming API.
    .RegisterMiddleware(openaiMessageConnector)
    .RegisterPrintMessage();

joe = new OpenAIChatAgent(
    openAIClient: openaiClient,
    name: "joe",
    modelName: openAIModel,
    systemMessage: """
    Your name is Joe and you are a stand-up comedian.
    When you're ready to end the conversation, say 'I gotta go'.
    End the conversation when you see two jokes.
    """)
    .RegisterMiddleware(tokenCountMiddleware)
    .RegisterMiddleware(openaiMessageConnector)
    .RegisterPrintMessage();

In [31]:
var chatHistory = new List<IMessage>
{
    new TextMessage(Role.User, "I'm Joe. Let's keep the jokes rolling.", from: joe.Name)
};
var roundLeft = 10;
while(roundLeft > 0)
{
    var replies = await joe.SendAsync(
        receiver: cathy,
        chatHistory,
        maxRound: 1);

    var reply = replies.Last();
    if (reply.GetContent()?.ToLower().Contains("i gotta go") is true)
    {
        break;
    }
    chatHistory.Add(reply);
    roundLeft--;
}

TextMessage from cathy
--------------------
Hey Joe! Glad you're ready for some laughs. Did you hear about the claustrophobic astronaut? He just needed a little space!
--------------------

TextMessage from joe
--------------------
Haha, that's a good one, Cathy! Here's one for you: Why couldn't the bicycle stand up by itself? Because it was two-tired!
--------------------

TextMessage from cathy
--------------------
Haha, I love it, Joe! Here's another one for you: Why did the math book look sad? Because it had too many problems!
--------------------

TextMessage from joe
--------------------
Haha, that's a good one, Cathy! Here's one for you: Why couldn't the leopard play hide and seek? Because he was always spotted!
--------------------

TextMessage from cathy
--------------------
Haha, that's a good one, Joe! Here's another one for you: Why did the scarecrow win an award? Because he was outstanding in his field!
--------------------

TextMessage from joe
--------------------
Haha, 